# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [ ]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/7/71/2010-kodiak-bear-1.jpg/1280px-2010-kodiak-bear-1.jpg
    
!wget https://www.zurbaran.com.ar/wp-content/uploads/2014/04/38099quinquela.jpg

In [ ]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

In [ ]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

#base_image_path = Path("/content/775px-Neckarfront_Tübingen_Mai_2017.jpg")
base_image_path = Path("/content/1280px-2010-kodiak-bear-1.jpg")
#style_reference_image_path = Path("/content/La_noche_estrellada1.jpg")
#style_reference_image_path = Path("/content/Ciclista.jpg")
style_reference_image_path = Path("/content/38099quinquela.jpg")
result_prefix = Path("/content/output")
iterations = 50 #100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:  

Cada uno de ellos es un peso de la loss:

**style_weight** = es el peso que tiene el estilo en el calculo de loss. Es el beta en la siguiente formula.

**content_weight** = es el peso que tiene el contenido en la loss. Es el alpha en la siguiente fomula.

![loss.png](loss.png)

**total_variation_weight** = peso de variacion de la imagen consigo misma, el objetivo de esta variable es mantener consistencia entre los pixeles de la imagen resultante.



# Sección nueva

In [ ]:
total_variation_weight = 0.1   
style_weight = 10 
content_weight = 1

In [ ]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size     #sube la imagen y calcula el tamaño
img_nrows = 400
img_ncols = int(width * img_nrows / height)       #calcula la cantidad de columnas a partir ancho por las filas dividido el alto
print (img_ncols)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

**Respuesta:**
Es una funcion donde recibe el path de la imagen, la carga convirtiendola en un array. En la anteultima sentencia,  le agrega una dimension quedando (1,img_nrows, img_ncols,3). 
Vgg19.preprocess_input(img): preprocesada el array, la cual es convertida de RGB a BGR, centrado en 0 respecto a imageNet, sin escalas.


In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))   ## carga la imagen ajustandola al tamaño calculado en la celda anterior
    img = img_to_array(img)  #convierte la imagen pil a un numpyp array
    img = np.expand_dims(img, axis=0)  #agrega una dimension al array, quedando (1,img_nrows, img_ncols,3)
    img = vgg19.preprocess_input(img)  #adecua la imagen al model vgg19
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

**Respuesta:** 
En la funcion Deprocess_image se revierten los cambios realizados en la función preprocess_image.

Es decir, quita la 4ta dismension agregada en el preprocess, descentra la imagen respecto al cero sumando la media a cada dimension, vuelve a convertirla a RGB y limita los valores entre 0 y 255. En la siguiente convierte a tensor las 3 variables.

In [ ]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939  #suma 103.939 a la dimension 0  Blue
    x[:, :, 1] += 116.779  #suma 116.779 a la dimension 1 Green
    x[:, :, 2] += 123.68  #suma 123.68 a la dimension 2  Red
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]    #invierte el orden de las dimensiones
    x = np.clip(x, 0, 255).astype('uint8')     # limita los valores del array entre 0 y 255 enteros
    return x

In [ ]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [ ]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [ ]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [ ]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?

Se utiliza para calcular la correlacion entre dos filtros o canales, calculando el producto punto entre los vectores de activacion de los mismos. Es utilizada en la formula de la style_loss. 

- ¿Por qué se permutan las dimensiones de x?

Por defecto los filtros estan en la 3er dimension, los pasa a la 1era dimension a fin de poder hacer el producto interno de estos en la Matriz de Gram y que queden bien ordenados al trasponer.


In [ ]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:

**Style_loss=**  mide la diferencia del estilo de la imagen de estilo y la generada. Cuanto menor es la loss mas parecido es el estilo entre estas imagenes.

**Content_loss=**  mide la diferencia entre el contenido de la imagen base y la imagen generada

**total_variation_loss =** es utilizada para calcular la coherencia local en la imagen generada, generando un transicion suave entre los pixeles, penalizando cambios abruptos.


In [ ]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [ ]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [ ]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [ ]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [ ]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:
Explique el propósito de las siguientes tres celdas. 

**eval_loss_and_grads:** La primer celda evalua la loss y el gradiente de la imagen ingresada.

**class Evaluator:** La segunda celda implementa la clase evaluator donde calcula la loss y el gradiente en forma conjunta.

**Generacion de imagenes:**  en la tercer celda se itera la cantidad de iteraciones definido en la variable iterations, tomando la imagen, calcula la minima loss de 20 iteraciones, deprocesa la imagen y la graba como una nueva imagen.


¿Qué hace la función fmin_l_bfgs_b? 
Minimiza la funcion definida en el primer parametro, en este caso la loss.

¿En qué se diferencia con la implementación del paper?
En esta implementación se utiliza la total_loss que no es nombrada en el paper. En el paper se usa la loss de style y de content.

¿Se puede utilizar alguna alternativa?
Se podria utilizar Adam o SGD en lugar de fmin_l_bfgs_b. 

In [ ]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [ ]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [ ]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta:


![oso.png](oso.png width=80px) 

**Imagen original e Imagen de Estilo:**

[<img src="OSO.png" width="500"/>](OSO.png)   [<img src="quinquela.jpg" width="500"/>](Quinquela.jpg)

**Parametros originales:**   total_variation_weight = 0.1   style_weight = 10  content_weight = 1

[<img src="\Quinquela - Oso\output\output_at_iteration_49.png" width="600"/>](output_at_iteration_49.png)

**1 - Parametros:**   total_variation_weight = 10   style_weight = 10  content_weight = 100

[<img src="\OSO_10_10_100\output_at_iteration_49.png" width="600"/>](output_at_iteration_49.png)

En esta configuración donde tiene mas peso el contenido que el resto de las variables, se puede apreciar que la imagen resultante esta mejor definida, con trazos mas suaves en las pinceladas, a pesar de que los colores se aproximan a la imagen de estilo.

**2 - Parametros:**   total_variation_weight = 10   style_weight = 100  content_weight = 10

[<img src="\OSO_10_100_10\output_at_iteration_49.png" width="600"/>](output_at_iteration_49.png)

En esta imagen el estilo es lo preponderante por lo cual los trazos de la pintura se pueden dislumbrar mas fuertemente, perdiendo en alguna medida la definicion del oso sobre todo en su cabeza.

**3 - Parametros:**   total_variation_weight = 100   style_weight = 10  content_weight = 10

[<img src="\OSO_100_10_10\output_at_iteration_49.png" width="600"/>](output_at_iteration_49.png)

Esta imagen resultante, pierde la nitidez, esto se debe a que el mayor peso esta dado en la total_variation_weight. El resultado es una imagen donde no se puede apreciar el estilo de la imagen de estilo ni el contenido de la imagen base. Quedando una imagen casi sin definiciones.

En todas las imagenes lo primero que pierde la imagen resultante es la gama de colores de la imagen original, asimilando la paleta de colores a la imagen de estilo.

Considero que la configuracion default del tp es la mas adecuada ya que mantiene una buena relacion entre estilo y contenido.

Todas las imagenes de las ejecuciones se encuentran en las carpetas adjuntas al trabajo practico, para el mismo se utilizaron 50 iteraciones.